# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. Sonika Bengani (2023aa05522)
2. Mugdha Hans (2023aa05570)
3. Vaibhav Bajpai(2023aa05631)


Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

Kindly refer supporting word document - ACI_Group_66_Assignment_1_PS6.docx

Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [142]:
#Code Block : Set Initial State (Must handle dynamic inputs)
import math
import time

class State:
#####################################################################################################
# Class     : State 
# Class Func: __init__
# Func Input: Current city, fare, time,h_cost(heuristic), f_cost, eval_cost 
# How?      : Initialises city state with fare, time, eval_cost,h_cost and f_cost
######################################################################################################    
    def __init__(self, city, fare, time, h_cost, f_cost):
        self.city = city
        self.g_cost = time    # Actual time from the start node
        self.h_cost = h_cost  # Heuristic time to the goal node
        self.f_cost = f_cost  # Total cost (f = g + h) in case of IDA* f = composite_eval_cost in\
                              # case of Hill_Climbing
        self.fare = fare      # Actual fare from the start node
        

In [143]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)
#Graph representing the problem statement 
graph = { # Dictionary of Dictionary
    0: {1: {"fare": 500, "time": 3, "ht": 3}, 2: {"fare": 650, "time": 2, "ht": 5}, 3: {"fare": 850, "time": 3, "ht": 1.5}},
    1: {0: {"fare": 500, "time": 3, "ht": 5}, 2: {"fare": 1000, "time": 4, "ht": 5}, 3: {"fare": 590, "time": 1.5, "ht": 1.5}, 5: {"fare": 1250, "time": 3, "ht": 0}},
    2: {0: {"fare": 650, "time": 2, "ht": 5}, 1: {"fare": 1000, "time": 4, "ht": 3}, 3: {"fare": 600, "time": 3, "ht": 1.5}},
    3: {0: {"fare": 850, "time": 3, "ht": 5}, 1: {"fare": 590, "time": 1.5, "ht": 3}, 2: {"fare": 600, "time": 3, "ht": 5}, 4: {"fare": 700, "time": 2, "ht": 1.5}, 5: {"fare": 1500, "time": 2, "ht": 0}},
    4: {3: {"fare": 700, "time": 2, "ht": 1.5}, 5: {"fare": 2500, "time": 3, "ht": 0}},
    5: {1: {"fare": 1250, "time": 3, "ht": 3}, 3: {"fare": 1500, "time": 2, "ht": 1.5}, 4: {"fare": 2500, "time": 3, "ht": 1.5}}
}

#Branghing factor calculation
b = max([len(graph[key]) for key in graph])
max_branch_factor = b - 1 #As one node will always be input by the user as a start node
#time_heuristics are maintained of each node. This is needed for IDA* implementation
time_heuristics = [5, 3, 5,1.5,1.5,0]

In [144]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented
######################################################################################################
# Function : Generate_Successors_IDAStar (Successor function of IDA* Algo)
# Input    : Current city state 
# Output   : List of successors 
# How ?    : Expands all the children of the Input city, their g_cost, h_cost and f_cost
#            time,as given in the graph Dictionary
######################################################################################################

def Generate_Successors_IDAStar(state,iteration):
    iteration += 1
    print("<ITERATION",iteration,"> IDA*") 
    successors = []
    print("Children of ",state.city)
    for child, cost in graph[state.city].items():
        g_cost =  state.g_cost + cost["time"]
        h_cost = cost["ht"]
        f_cost = g_cost + time_heuristics[child]
        successor = State(child, 0, g_cost, h_cost,f_cost)
        print("{Child ",successor.city, ", time:",successor.g_cost, \
              ",h_time:",successor.h_cost, ", Evaluation Cost:",successor.f_cost,"}")
        successors.append(successor)
    return successors

#####################################################################################################
# Function : Composite_Eval_Cost 
# Input    : Fare and time of Current city 
# Output   : Returns the evaluation cost 
# How ?    : Accounting the weightage of fare(wf) and time(wt) needed for HillClimbingAlgo 
# Note     : Fare is log transformed to bring both on the same scale
##################################################################################################### 
def Composite_Eval_Cost(fare,time):
    if fare > 0:
        e = wf*(math.log10(abs(fare))) + wt*(abs(time))
    return e

######################################################################################################
# Function : Generate_Successors_HillClimbing (Successor function of HillClimbingAlgo)
# Input    : Current city state 
# Output   : List of successors 
# How ?    : Expands all the children of the Input city, their fare,
#            time,as given in the graph Dictionary
######################################################################################################      
def Generate_Successors_HillClimbing(state,iteration):
    
    iteration += 1
    print("<ITERATION",iteration,"> Hill-Climbing") 
    successors = []
    print("Children of",state.city)
    for child, cost in graph[state.city].items():
      #If child not already in expanded node, only then initialise
      if child not in expanded_node:
        fare =  cost["fare"]
        time =  cost["time"]
        h_cost = cost["ht"]
        f_cost = Composite_Eval_Cost(fare,time) #Evalcost 
        successor = State(child, fare, time, h_cost,f_cost)
        print("{Child ",successor.city, ", fare:",successor.fare,", time:",\
              successor.g_cost, ", Evaluation Cost:",successor.f_cost,"}")
        successors.append(successor)
    return successors



In [145]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented
########################################################################
# Function : Goal_Test
# Input    : Current city state
# Output   : status of goal test(True/False)
# How      : Compares the city at hand with goal city
#            
#########################################################################
def Goal_Test(state):
    return state.city == goal

### 2.	Definition of - IDA*

In [146]:
#Code Block : Function for IDA* implementation
import math

#########################################################################
# Function : Select_Min_fscore_Child 
# Input    : List of successors of expanded node 
# Output   : Child with minimum evaluation cost
# How ?    : Compare eval_cost of all given neighbors
######################################################################### 
def Select_Min_fscore_Child(successors,limit):
  min_child = None  
  for child in successors:
    
    if(child.f_cost <= limit):
      min_fscore = child.f_cost
      min_child = child
      print("Min Child=",min_child.city)
      
    else:
        pruned_node.append(child)
  return min_child

#########################################################################
# Function : Change_Limit 
# Input    : List of successors of expanded node 
# Output   : Child with minimum evaluation cost
# How ?    : Compare eval_cost of all given neighbors
######################################################################### 
def Change_Limit(pruned_node):
    min_fscore = float('inf')
    for i in range(len(pruned_node)):
        if(pruned_node[i].f_cost < min_fscore):
            new_limit = pruned_node[i].f_cost
            min_fscore = new_limit
        
    pruned_node = []
    return new_limit
        
###############################################################################################
# Function : IDA_Star_Algorithm 
# Input    : Current city state 
# Output   : Optimal path
# How ?    : Step 1 - Generate Successors
#            Step 2 - Select child with min f_cost, if not found - change the limit
#            Step 3 - Perform Goal test on min_child
#            Step 4 - If goal test fails IDA* recursively
############################################################################################### 
def IDA_Star_Algorithm(state,limit,iteration):
    
#Generates successors of state in hand
    successors = Generate_Successors_IDAStar(state,iteration)
  
    if successors: 
        min_fn_state = Select_Min_fscore_Child(successors,limit)
        if min_fn_state:
            if Goal_Test(min_fn_state):
                print("***Eureka!!! Found Goal!!****")
                return "Goal"
            else:
                expanded_node.append(min_fn_state.city)
                result = IDA_Star_Algorithm(min_fn_state,limit,iteration)
                if(result == "Goal"):
                    expanded_node.append(goal)
                    print("Traversed Node:",expanded_node)
        else:
            new_limit = Change_Limit(pruned_node)
            print("New limit",new_limit)
            limit = new_limit
            IDA_Star_Algorithm(state,limit,iteration)
        

### 3.	Definition of - HillClimbing

In [147]:
#Code Block : Function for HillClimbing implementation
#########################################################################
# Function : Select_Min_Eval_Child 
# Input    : List of successors of expanded node 
# Output   : Child with minimum evaluation cost
# How ?    : Compare eval_cost of all given neighbors
######################################################################### 
def Select_Min_Eval_Child(successors):
  min_fn = float('inf')
  for child in successors:
    if(child.f_cost < min_fn):
      min_fn = child.f_cost
      min_child = child
  return min_child

#########################################################################
# Function : Hill_Climbing_Algo 
# Input    : Current city state 
# Output   : Returns Goal if foudn
# How ?    : Step 1 - Generate Successors
#            Step 2 - Select child with min evaluation cost
#            Step 3 - Perfom Goal test on min_child
#            Step 4 - If goal test fails hill climb recursively
######################################################################### 
def Hill_Climbing_Algo(state,iteration):
   
    #Generates successors of state in hand
    successors = Generate_Successors_HillClimbing(state,iteration)
    if successors:
        min_fn_state = Select_Min_Eval_Child(successors)
        print("Select min fn city:",min_fn_state.city)
        if Goal_Test(min_fn_state):
            print("*****Eureka!!!!!!Goal Found!*****")
            return "Goal"
        else:
            expanded_node.append(min_fn_state.city)
            result = Hill_Climbing_Algo(min_fn_state,iteration)
            if(result == "Goal"):
                expanded_node.append(goal)
      
    else:
        print("!!!!!Stuck at city",state.city,", Could not expand further!!!!!")


### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [148]:
#Code Block : Function & call to get inputs (start/end state)
print("Choose Algorithm")
print("1 - Iterative Deepening A Star Algorithm")
print("2 - HillClimbing")

choice = int(input("Enter choice(1/2)-"))
print("********Mention start and goal city**********")
root = int(input("Enter start city index(0 for A):"))
goal = int(input("Enter goal city index(5 for G):"))

if(choice == 1):
#Set limit for IDA*
    limit = time_heuristics[root]
    new_limit = 0
    
if(choice == 2):
    print("\nWe are using composite Evaluation function to calculate f-cost which takes weightage of fare and time from the user")
    print("Also, for the given problem statement,")
    print("Please input start city index = 0 and goal city index = 5")
    
    print("Please give weightage values for fare(wf) and time(wt)")
    print("Please note wf+wt = 1")

    #Take input from user for weightage of fare. Weightage of time will be calculated
    wf = float(input("What weightage you want to give to fare?(should be < 1 for e.g 0.8)"))
    wt = 1-wf # As we know that wf+wt =1
    


#Expanded node list keeps track of cities whose children are generated
expanded_node = []
#Pruned node list keeps track of cities whose f-score is > limit
pruned_node = []

#Create instance of State class for root node
root_state = State(root, 0, 0, time_heuristics[root], 0) 

#Expand Root Node
expanded_node.append(root_state.city)



Choose Algorithm
1 - Iterative Deepening A Star Algorithm
2 - HillClimbing
Enter choice(1/2)-2
********Mention start and goal city**********
Enter start city index(0 for A):2
Enter goal city index(5 for G):5

We are using composite Evaluation function to calculate f-cost which takes weightage of fare and time from the user
Also, for the given problem statement,
Please input start city index = 0 and goal city index = 5
Please give weightage values for fare(wf) and time(wt)
Please note wf+wt = 1
What weightage you want to give to fare?(should be < 1 for e.g 0.8)0.7


### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [132]:

from math import sqrt

#Invoke algorithm 1/2 (Should Print the solution, path, cost etc., (As mentioned in the problem))

#################################################################################################
# Function : Calc_Optimal_Cost 
# Input    : Null 
# Output   : Prints Optimal Fare and Time of Optimal path found
# How ?    : Calculates optimal fare and time from expanded_node list
################################################################################################# 
def Calc_Optimal_Cost(choice):
  optimal_fare = 0
  optimal_time = 0
  for i in range(len(expanded_node) - 1):
    cost = graph[expanded_node[i]][expanded_node[i+1]]
    optimal_fare += cost["fare"]
    optimal_time += cost["time"]
  print("Optimal Fare:",optimal_fare)
  print("Optimal Time:",optimal_time)

    
iteration = 0

#If root city is not goal city, then call appropriate algo as per user choice
if not Goal_Test(root_state):
    if choice == 1:
        start_time = time.time()
        IDA_Star_Algorithm(root_state,limit,iteration)
        end_time = time.time() 
    if choice == 2 :
        start_time= time.time()
        result = Hill_Climbing_Algo(root_state,iteration)
        end_time = time.time()

 
#The expanded nodes represent the optimal path
print("Optimal Path:",expanded_node)
Calc_Optimal_Cost(choice)

<ITERATION 1 > Hill-Climbing
Children of 2
{Child  0 , fare: 650 , time: 2 , Evaluation Cost: 2.6503306853142847 }
{Child  1 , fare: 1000 , time: 4 , Evaluation Cost: 3.2 }
{Child  3 , fare: 600 , time: 3 , Evaluation Cost: 2.8225210003069146 }
Select min fn city: 0
<ITERATION 1 > Hill-Climbing
Children of 0
{Child  1 , fare: 500 , time: 3 , Evaluation Cost: 2.7591760034688146 }
{Child  3 , fare: 850 , time: 3 , Evaluation Cost: 2.9435351405714343 }
Select min fn city: 1
<ITERATION 1 > Hill-Climbing
Children of 1
{Child  3 , fare: 590 , time: 1.5 , Evaluation Cost: 2.516681609313715 }
{Child  5 , fare: 1250 , time: 3 , Evaluation Cost: 3.0775280104064446 }
Select min fn city: 3
<ITERATION 1 > Hill-Climbing
Children of 3
{Child  4 , fare: 700 , time: 2 , Evaluation Cost: 2.6760784320114057 }
{Child  5 , fare: 1500 , time: 2 , Evaluation Cost: 2.9408730072445453 }
Select min fn city: 4
<ITERATION 1 > Hill-Climbing
Children of 4
{Child  5 , fare: 2500 , time: 3 , Evaluation Cost: 3.318352

### 5.	Comparitive Analysis

In [133]:
#Code Block : Print the Time & Space complexity of algorithm IDA*
if choice == 1 :
    print("Time Complexity of IDA* in millisec :", (end_time - start_time)*1000,"ms")
    #Space complexity of IDA* is O(bxd)
    space_comp = max_branch_factor*(len(expanded_node) - 1)
    print("Space Complexity of IDA* :",space_comp)


In [134]:
#Code Block : Print the Time & Space complexity of algorithm Hill Climbing
if choice == 2 :
    print("Time complexity of Hill Climbing in millisec:", (end_time - start_time)*1000,"ms")
    #Space complexity of Hill Climbing is O(b)
    print("Space Complexity of Hill Climbing :",max_branch_factor)

Time complexity of Hill Climbing in millisec: 3.0007362365722656 ms
Space Complexity of Hill Climbing : 4


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : 
IDA* has higher space complexity as compared to Hill Climbing. The time complexity of both algorithms depends on problem characteristics and choice of heuristic. Our implementation of Hill Climbing uses  weighted sum of time and fare. For the default problem statement both algorithms are complete and giving optimal solution.

________________________________________________________

_________________________________________________________